## Constructing Tasks from Bash Scripts

Run a Bash script from within a workflow.

### Prerequisites

Import `covalent`.

In [1]:
import covalent as ct

Now let's invoke a generic Bash command instead. As an example, we'll return the current version of the gcc compiler. More advanced users may wish to invoke a Makefile or a build command for software on a remote backend machine.

In [ ]:
task = ct.Lepton(
    language="bash",
    command="version=`{CPP} --version | awk 'NR==1 {{print $3}}'` \
        && IFS=. read major minor patch <<< $version \
    ",
    argtypes=[
        (str, ct.Lepton.INPUT_OUTPUT),
        (int, ct.Lepton.OUTPUT),
        (int, ct.Lepton.OUTPUT),
        (int, ct.Lepton.OUTPUT)
    ],
    named_outputs=["CPP", "major", "minor", "patch"]
)

Run the task and print the result:

In [ ]:
@ct.lattice
def version_workflow(**kwargs) -> str:
    return task(**kwargs)

result = ct.dispatch_sync(version_workflow)(CPP="gcc").result
print(result)

This results from parsing the output of `gcc --version`,  which on my system returns

```
gcc (GCC) 11.2.1 20220127 (Red Hat 11.2.1-9)
Copyright (C) 2021 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
```
The `named_outputs` parameter tells Covalent the environment variables from which to read the script output. We must also specify the corresponding types in the `argtypes` array when constructing the Lepton. In this example, the gcc major version, minor version, and patch version are saved to the `read`, `minor`, and `patch` env variables respectively, and each is to be interpreted as a Python `int`.